### Calculating Solar Elevation from NEON Filenames

In order to lower the disk space usage of NEON images on the PhenoCam server we want
to eliminate the nighttime images from NEON.  The proposed action 
is to only archive images between nautical twighlights (sun 12 degrees below the horizon).

Here's an example NEON filename as transferred to UNH:

    NEON.D02.SERC.DP1.00033.01797_2016_02_29_213005.jpg
    
The relevant pieces of the filename for calculating the solar elevation are the site-id, `SERC`,
(which can be used to look up the lat and lon of the site) and the date and time.  For the `SERC`
site the lat and lon values are 38.8901 and -76.5600.  At unh we use the python `ephem` package to
calculate the solar elevation as shown below.  

The notebook below details the calculations done at UNH.

In [1]:
import os
import sys
from datetime import datetime
from datetime import timedelta
import ephem

In [2]:
def deg2dms(angle):
    """
    helper function to convert angle in fractional
    degrees to a string of the form dd:mm:ss
    """
    deg = int(angle)
    degdiff = abs(angle - float(deg))
    minangle = degdiff * 60.
    min = int(minangle)
    mindiff =  minangle - float(min) 
    sec = int(mindiff * 60.)
    dmsstr = "{0:02d}:{1:02d}:{2:02d}".format(deg,min,sec)
    return dmsstr

In [3]:
def sunelev_utc(lat, lon, utcdt):
    """
    function to return the solar elevation at a given
    latitude (decimal degrees) and longitude (decimal degrees)
    and UTC date and time.  
    """

    # set up observer for ephem package
    site = ephem.Observer()
    site.lat = deg2dms(lat)
    site.lon = deg2dms(lon)
    site.date = utcdt

    # get sun position
    sun = ephem.Sun(site)

    # return elevation is decimal degrees
    elev = sun.alt/ephem.degree

    return elev

In [4]:
filename = "NEON.D02.SERC.DP1.00033.01797_2016_02_29_213005.jpg"
lat = 38.8901
lon = -76.5600

# extract UTC date and time from filename
bname, ext = os.path.splitext(filename)
fbase, year, month, dom, hhmmss = bname.split('_')
utcdt = datetime.strptime(year+month+dom+hhmmss, "%Y%m%d%H%M%S")
print(utcdt)

2016-02-29 21:30:05


In [5]:
elev = sunelev_utc(lat, lon, utcdt)
print(elev)

15.683449636332014


In [6]:
# define a function which takes a filename, lat and lon and
# spits out the solar elevation
def fn2se(filename, lat, lon):
    """
    calculate solar elevation from NEON filename and lat, lon
    """
    # extract UTC date and time from filename
    bname, ext = os.path.splitext(filename)
    fbase, year, month, dom, hhmmss = bname.split('_')
    utcdt = datetime.strptime(year+month+dom+hhmmss, "%Y%m%d%H%M%S")
   
    elev = sunelev_utc(lat, lon, utcdt)
    
    return utcdt, elev

In [7]:
utcdt, solar_elev = fn2se(filename, lat, lon)
print(utcdt, solar_elev)

2016-02-29 21:30:05 15.683449636332014


In [8]:
# try this on a day's worth of images
filelist = ["NEON.D02.SERC.DP1.00033.01797_2016_02_29_000006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_001506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_003005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_004505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_010005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_011505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_013005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_014505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_020005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_021505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_023005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_024505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_030005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_031506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_033006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_034506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_040006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_041506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_043006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_044506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_050005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_051505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_053005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_054505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_060005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_061505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_063005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_064506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_070006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_071505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_073006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_074506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_080006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_081506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_083006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_084505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_090005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_091506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_093006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_094506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_100006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_101505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_103005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_104505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_110005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_111505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_113006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_114505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_120006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_121506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_123006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_124506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_130006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_131505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_133006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_134505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_140006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_141506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_143005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_144505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_150006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_151506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_153005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_154505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_160005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_161505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_163006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_164505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_170006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_171505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_173005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_174505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_180006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_181506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_183006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_184505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_190006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_191505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_193005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_194506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_200005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_201506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_203005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_204505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_210006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_211506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_213005.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_214506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_220006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_221506.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_223006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_224505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_230006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_231505.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_233006.jpg",
"NEON.D02.SERC.DP1.00033.01797_2016_02_29_234506.jpg"]


In [9]:
# lst offset from utc
utc_offset = -5

# setup some counters
nkeep = 0
ndelete = 0
nimages = len(filelist)

print("filename,local_standard_datetime,solar_elevation,action")

# loop over list and decide an action based on nautical twighlight (solar elevation = -12)
for fname in filelist:
    utc_dt, solar_elev = fn2se(fname, lat, lon)
    utc_lst = utc_dt + timedelta(hours=utc_offset)
    lst_dt = utc_lst.strftime("%Y-%m-%d %H:%M:%S")
    if solar_elev < -12:
        action = "discard"
        ndelete += 1
    else:
        action = "archive"
        nkeep += 1
    print("{},{},{:.1f},{}".format(fname, lst_dt, solar_elev, action))
    
print("Total number of images: {}".format(nimages))
print("Number archived: {}".format(nkeep))
print("Number discarded: {}".format(ndelete))

filename,local_standard_datetime,solar_elevation,action
NEON.D02.SERC.DP1.00033.01797_2016_02_29_000006.jpg,2016-02-28 19:00:06,-13.0,discard
NEON.D02.SERC.DP1.00033.01797_2016_02_29_001506.jpg,2016-02-28 19:15:06,-15.9,discard
NEON.D02.SERC.DP1.00033.01797_2016_02_29_003005.jpg,2016-02-28 19:30:05,-18.8,discard
NEON.D02.SERC.DP1.00033.01797_2016_02_29_004505.jpg,2016-02-28 19:45:05,-21.7,discard
NEON.D02.SERC.DP1.00033.01797_2016_02_29_010005.jpg,2016-02-28 20:00:05,-24.6,discard
NEON.D02.SERC.DP1.00033.01797_2016_02_29_011505.jpg,2016-02-28 20:15:05,-27.4,discard
NEON.D02.SERC.DP1.00033.01797_2016_02_29_013005.jpg,2016-02-28 20:30:05,-30.3,discard
NEON.D02.SERC.DP1.00033.01797_2016_02_29_014505.jpg,2016-02-28 20:45:05,-33.1,discard
NEON.D02.SERC.DP1.00033.01797_2016_02_29_020005.jpg,2016-02-28 21:00:05,-35.8,discard
NEON.D02.SERC.DP1.00033.01797_2016_02_29_021505.jpg,2016-02-28 21:15:05,-38.5,discard
NEON.D02.SERC.DP1.00033.01797_2016_02_29_023005.jpg,2016-02-28 21:30:05,-41.1,discar